# Integrating v2

In [1]:
%load_ext autoreload
%autoreload 2

In [56]:
import os
import subprocess
import yaml

from astropy.io import fits
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
import random

from scipy.interpolate import interp2d, RectBivariateSpline
from matplotlib import pyplot as plt

In [4]:
from gravitational_wave_toy import GWToyV2 as gwt

/Users/jarred/Documents/Projects/gravitational_wave_toy/.venv/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


## Import a sample grb

In [5]:
filepath = "./GammaCatalogV2.0/run0639_ID000222.fits"

In [350]:
grb = gwt.GRB(filepath)

In [289]:
time_index = 50
energy_index = 10
print(f"Spectrum at t={grb.time[time_index]}s:")
print(np.array(grb.lc[time_index]))
print()
print(f"Value at {grb.energy[energy_index]} GeV:")
print(np.array(grb.lc[time_index])[energy_index])
print()
print(f"Lightcurve at energy {grb.energy[energy_index]} GeV:")
print(grb.lc.field(10))
print()
print(f"LC value at t={grb.time[time_index]}s:")
print(grb.lc.field(10)[time_index])

Spectrum at t=10000.0s:
[8.63502424e-18 5.27549920e-18 3.22302414e-18 1.96908039e-18
 1.20299344e-18 7.34959249e-19 4.49017492e-19 2.74323656e-19
 1.67595839e-19 1.02391059e-19 6.25542882e-20 3.82155421e-20
 2.33474807e-20 1.42639571e-20 8.71444111e-21 5.32392314e-21
 3.25222328e-21 1.98523708e-21 1.21286476e-21 7.40985362e-22
 4.52678923e-22 2.76444973e-22 1.68380404e-22 1.02708634e-22
 6.27439278e-23 3.83145104e-23 2.33234840e-23 1.40566756e-23
 8.58121812e-24 5.22333427e-24 3.07889038e-24 1.87396066e-24
 1.11737198e-24 6.55875186e-25 3.75313085e-25 2.20153637e-25
 1.22507256e-25 6.47949218e-26 2.60612051e-26 4.39645129e-27
 0.00000000e+00]

Value at 10.0 GeV:
6.255428820326217e-20

Lightcurve at energy 10.0 GeV:
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.77748540e-27 1.59431806e-26 

In [299]:
grb.get_spectrum(time=10)

array([1.55617664e-23, 9.50733707e-24, 5.80843172e-24, 3.54861402e-24,
       2.16799640e-24, 1.32452011e-24, 8.09205000e-25, 4.94377341e-25,
       3.02035872e-25, 1.84525845e-25, 1.12733168e-25, 6.88704282e-26,
       4.20758388e-26, 2.57058982e-26, 1.57047940e-26, 9.59452848e-27,
       5.86093189e-27, 3.57732648e-27, 2.18553908e-27, 1.33522831e-27,
       8.15706748e-28, 4.98118456e-28, 3.03300997e-28, 1.84976275e-28,
       1.12999599e-28, 6.89994938e-29, 4.19860618e-29, 2.52664027e-29,
       1.54231285e-29, 9.38409586e-30, 5.50909135e-30, 3.35164106e-30,
       1.99269236e-30, 1.16399745e-30, 6.60339383e-31, 3.85218069e-31,
       2.11525897e-31, 1.09137318e-31, 4.01687486e-32, 3.72658767e-33,
       0.00000000e+00])

In [296]:
grb.get_lightcurve(energy=10)

array([ 1.48468592e-49, -8.57677536e-49,  2.83674822e-48, -7.87484184e-48,
        2.10621376e-47, -5.60769423e-47,  1.49287273e-46, -3.97846373e-46,
        1.05927011e-45, -2.81764994e-45,  7.46766184e-45, -1.98371314e-44,
        5.29865982e-44, -1.42582119e-43,  3.89560973e-43, -1.23314265e-42,
        5.77748540e-27,  1.59431806e-26,  3.56370279e-26,  6.49518841e-26,
        1.12733168e-25,  1.90125986e-25,  3.02651059e-25,  4.67179983e-25,
        7.17603785e-25,  1.11239107e-24,  1.74451275e-24,  2.70119487e-24,
        4.16377708e-24,  6.39565046e-24,  9.91419588e-24,  1.55479862e-23,
        2.40744246e-23,  3.71097023e-23,  5.70012948e-23,  8.83603638e-23,
        1.38571573e-22,  2.14563535e-22,  3.30740570e-22,  5.08024574e-22,
        7.87512571e-22,  1.23502045e-21,  1.91229952e-21,  2.94772843e-21,
        4.52777378e-21,  7.01871317e-21,  1.10071313e-20,  1.70433872e-20,
        2.62716558e-20,  4.03538175e-20,  6.25542882e-20,  9.81007526e-20,
        1.51896850e-19,  

In [351]:
grb.__dict__.keys()

dict_keys(['zenith', 'site', 'seen', 'obs_time', 'start_time', 'end_time', 'run', 'id', 'ra', 'dec', 'eiso', 'z', 'angle', 'time', 'energy', 'min_energy', 'max_energy', 'spectra', 'spectrum', 'rng'])

In [353]:
keys_to_drop = ["time", "energy", "spectra", "spectrum", "rng"]
output = { key : value for key, value in grb.__dict__.items() if key not in keys_to_drop }
    

In [354]:
output

{'zenith': 40,
 'site': 'north',
 'seen': False,
 'obs_time': -1,
 'start_time': -1,
 'end_time': -1,
 'run': '0639',
 'id': '000222',
 'ra': 36.554,
 'dec': 46.013,
 'eiso': 6.17e+48,
 'z': 0.021,
 'angle': 89.93,
 'min_energy': 1.0,
 'max_energy': 10000.0}